In [1]:
import pandas as pd
import sklearn.model_selection as ms
from sklearn.linear_model import Ridge
import sklearn.metrics as me

In [2]:
RANDOM_STATE = 42

In [3]:
flights_engi = pd.read_csv('../Data/files/flights_engi.csv')
flights_engi.head()

,branded_code_share,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,distance,arr_delay,origin_airport_id_rank,dest_airport_id_rank,month_of_year_rank,day_of_week_rank
0,1,1020,1112,0,52.0,127,-2.0,205.0,62.0,8.0,4.0
1,1,1148,1340,0,112.0,468,-14.0,194.0,196.0,8.0,4.0
2,1,1155,1551,0,176.0,1091,4.0,205.0,235.0,8.0,4.0
3,1,839,959,0,80.0,310,-20.0,138.0,182.0,8.0,4.0
4,1,710,826,0,76.0,301,-1.0,290.0,196.0,8.0,4.0


Since there are so many observations, let us start with a pseudorandom subsample to check computational time and scale accordingly.

In [4]:
# 15% of the data.
flights_subsample = flights_engi.sample(frac=0.15, random_state=RANDOM_STATE)

In [5]:
X = flights_subsample.drop('arr_delay', axis=1)
y = flights_subsample['arr_delay']

### Processing the data

In [6]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, random_state=RANDOM_STATE, test_size=0.3)

In [7]:
ridge = Ridge(random_state=RANDOM_STATE)

In [8]:
ridge.fit(X_train, y_train)

Ridge(random_state=42)

In [9]:
y_pred = ridge.predict(X_test)

In [10]:
me.mean_squared_error(y_test, y_pred)

2379.3844377874098

In [11]:
me.r2_score(y_test, y_pred)

0.019990681538700095

In its current state, our model is worse than terrible. That's okay. This is where the iteration properly begins. The data is horrible right now, so fetures need to be significantly adjusted. The model choice is currently arbitrary, and the hyperparameters are default.